# 2019 MolSSI Summer School QM project: semiempirical model of Argon

## 1. Introduction

In this project, we will simulate a cluster of Argon atoms using quantum mechanics (QM) to calculate their total energy. First-principles QM simulations are complicated and expensive, and a quick implementation would rely on a substantial amount of pre-existing software infrastructure. Instead, we will implement a much simpler semiempirical QM simulation that has been designed and parameterized to reproduce first-principles QM data using a minimal model. We can then limit our external dependencies to the standard mathematical functionality of Python: NumPy to build and solve matrix and tensor equations and SymPy to compute analytical derivatives as needed:

In [1]:
import numpy as np
import sympy

As is typically the case in quantum chemistry, we will treat the atomi nuclei as classical point charges,

$$ \vec{r}_i = (x_i , y_i, z_i). $$

While we would normally read these from a file or pass them in as an argument to a function, we will just specify a set of atomic coordinates in this example:

In [4]:
atomic_coordinates = np.array([ [0.0,0.0,0.0], [3.0,4.0,5.0] ])
number_of_atoms = len(atomic_coordinates)

np.set_printoptions(precision=1)
print(atomic_coordinates)

[[0. 0. 0.]
 [3. 4. 5.]]


Similarly, we will only calculate and print the total energy to the screen, whereas more useful software would probably print output data to a file or return it from a function call. You will add such features as you refactor this software. All physical quantities in this project will be specified in Hartree atomic units, where the bohr is the unit of length and the hartree is the unit of energy.

## 2. Model Hamiltonian

This semiempirical model will combine some standard concepts and methods used in physics and chemistry. First, it will use a minimal number of electronic degrees of freedom. Because Argon is a noble gas, it interacts primarily through London dispersion forces that are mediated by quantum dipole fluctuations. The lowest energy dipole transition is from the occupied $3p$ states to the unoccupied $4s$ state, and we will include these 4 atomic orbitals per atom. Similarly, we will use a multipole expansion to simplify electronic excitations and retain only the monopole and dipole terms, which also restricts electronic polarization to 4 degrees of freedom per atom. We will use $\{s, p_x, p_y, p_z\}$ to label both sets of basis functions on each atom.

In [6]:
orbital_types = ['s', 'px' ,'py', 'pz']
orbitals_per_atom = len(orbital_types)
ndof = number_of_atoms*orbitals_per_atom

p_orbitals = orbital_types[1:]
print(p_orbitals)

['px', 'py', 'pz']


Within our minimal basis, we will describe the electronic state as the ground state of a quantum many-body Hamiltonian $\hat{H}$. As is standard in quantum chemistry, we begin by writing the Hamiltonian in second quantization notation,

$$ \hat{H} = E_{\mathrm{ion}} + \sum_{p,q} \sum_{\sigma \in \{ \uparrow , \downarrow \} } h_{p,q} \hat{a}_{p,\sigma}^{\dagger} \hat{a}_{q,\sigma} + \tfrac{1}{2}\sum_{p,q,r,s} \sum_{\sigma,\sigma' \in \{ \uparrow , \downarrow \} } V_{p,q,r,s} \hat{a}_{p,\sigma}^{\dagger} \hat{a}_{r,\sigma'}^{\dagger} \hat{a}_{s,\sigma'} \hat{a}_{q,\sigma} , $$

where $\hat{a}_{p,\sigma}^{\dagger}$ and $\hat{a}_{p,\sigma}$ are the electron raising and lowering operators for an atomic orbital index $p$ and spin $\sigma$. We will not be using $\hat{H}$ itself in our calculations, but we will make use of the coefficient tensors $h_{p,q}$ and $V_{p,q,r,s}$. In first-principles calculations, each element of $h_{p,q}$ and $V_{p,q,r,s}$ would require the evaluation of a complicated integral. In our semiempirical model, we will set most of them to zero and assign a simple analytical form to the rest of them. We will also assume that the atomic orbitals are all orthogonal, which avoids having to deal with overlap matrices. The notation being used here is mostly consistent with modern quantum chemistry notation, but some objects, particularly $V_{p,q,r,s}$, have multiple conventions in practice.

The atomic orbital index contains information about which atom the orbital is centered on and the type of orbital it is. We will often extract these individual pieces of information using $\mathrm{atom}(p)$ to denote the atom and $\mathrm{orb}(p)$ to denote the orbital type. This is the first of many instances in this project where we could either represent something as a pre-tabulate list or a function. We will always make the simpler choice, in this case functions:

In [39]:
def atom(ao_index):
    """Return the atom index part of an atomic orbital index."""
    return ao_index // orbitals_per_atom

def orb(ao_index):
    """Return the orbital type of an atomic orbital index."""
    orb_index = ao_index % orbitals_per_atom
    return orbital_types[orb_index]

def ao_index(atom_p,orb_p):
    """Return the atomic orbital index for a given atom index and orbital type."""
    p = atom_p*orbitals_per_atom
    p += orbital_types.index(orb_p)
    return p

print("1st atom index =", atom(0))
print("1st orbital type =", orb(0))
print("ao index of s orbital on 1st atom =", ao_index(0, 's'))

1st atom index = 0
1st orbital type = s
ao index of s orbital on 1st atom = 0


### A. Coulomb interaction

The nuclear charge of Argon is 18, but our model combines the nucleus and the 12 neglected electrons ($1s^2$, $2s^2$, $2p^6$, and $3s^2$) as an ionic point charge with $Z = 6$. For the purpose of electrostatics, we will describe the valence electrons of each Argon atom as Gaussian charges and their derivatives, which form a multipole expansion. The width of these Gaussians, $r_0$, is the first parameter of our semiempirical model:

In [24]:
ionic_charge = 6.0
r0 = 6.5

The electron-ion and electron-electron interactions have the same functional form, but the electron-electron interaction has an wider effective width of $\sqrt{2}r_0$,

$$ V^{\mathrm{II}}(r) = \frac{Z^2}{r} \\
   V^{\mathrm{eI}}(r) = -Z \frac{\mathrm{erf}(r/r_0)}{r} \\
   V^{\mathrm{ee}}(r) = \frac{\mathrm{erf}(r/(\sqrt{2}r_0))}{r}. $$

These functions can be constructed using SymPy, which will allow us to evaluate analytical derivatives automatically:

In [40]:
x1, y1, z1, x2, y2, z2 = sympy.symbols('x1 y1 z1 x2 y2 z2')
r = sympy.sqrt( (x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2 + 1e-16)

v_ii_kernel = ionic_charge**2 / r
v_ei_kernel = -ionic_charge*sympy.erf( r / r0 ) / r
v_ee_kernel = sympy.erf( r / ( np.sqrt(2.0) * r0 ) ) / r

def eval_kernel(kernel):
    """Return a NumPy function that efficiently evaluates the SymPy function input."""
    return sympy.lambdify(([x1,y1,z1], [x2,y2,z2]), kernel)

It is then straightforward (but slow) to "compile" a function from SymPy to NumPy to evaluate formulas. For example, the ion-ion energy in $\hat{H}$,

$$ E_{\mathrm{ion}} = \sum_{i < j} V^{\mathrm{II}}(|\vec{r}_i - \vec{r}_j|) $$

can be implemented as:

In [41]:
def calculate_energy_ion():
    """Return the ionic contribution to the total energy."""
    energy_ion = 0.0
    v_ii = eval_kernel( v_ii_kernel )
    for i, r_i in enumerate(atomic_coordinates):
        for j, r_j in enumerate(atomic_coordinates):
            if i < j:
                energy_ion += v_ii(r_i, r_j)
    return energy_ion

energy_ion = calculate_energy_ion()
print(energy_ion)

5.091168824543142


For functions involving electrons, we need to define a differential operator that will generate our multipole expansions,

$$ \Delta_p = \begin{cases} 1 , & \mathrm{orb}(p) = s \\
d/d x_{\mathrm{atom}(p)} , & \mathrm{orb}(p) = p_x \\
d/d y_{\mathrm{atom}(p)} , & \mathrm{orb}(p) = p_y \\
d/d z_{\mathrm{atom}(p)} , & \mathrm{orb}(p) = p_z
\end{cases}, $$

In [42]:
def grad1(kernel,orb1):
    """Return the input interaction kernel with a multipole generator in the 1st coordinate applied to it."""
    if orb1 == 's':
        return kernel
    if orb1 == 'px':
        return sympy.diff(kernel, x1)
    if orb1 == 'py':
        return sympy.diff(kernel, y1)
    if orb1 == 'pz':
        return sympy.diff(kernel, z1)

def grad2(kernel,orb2):
    """Return the input interaction kernel with a multipole generator in the 2nd coordinate applied to it."""
    if orb2 == 's':
        return kernel
    if orb2 == 'px':
        return sympy.diff(kernel, x2)
    if orb2 == 'py':
        return sympy.diff(kernel, y2)
    if orb2 == 'pz':
        return sympy.diff(kernel, z2)

both for the vector of electron-ion potential matrix elements,

$$ V^{\mathrm{eI}}_p = \sum_{i} \Delta_p V^{\mathrm{eI}}(|\vec{r}_{\mathrm{atom}(p)} - \vec{r}_i|) , $$

In [43]:
def calculate_potential_vector():
    """Return the electron-ion potential energy vector."""
    potential_vector = np.zeros(ndof)
    for orb_p in orbital_types:
        v_ei = eval_kernel( grad1(v_ei_kernel, orb_p) )
        for atom_p,r_p in enumerate(atomic_coordinates):
            p = index(atom_p,orb_p)
            for r_i in atomic_coordinates:
                potential_vector[p] += v_ei(r_p, r_i)
    return potential_vector

potential_vector = calculate_potential_vector()
print(potential_vector)

[-1.8 -0.  -0.  -0.  -1.8  0.   0.   0. ]


and the matrix of electron-electron interaction matrix elements,

$$ V^{\mathrm{ee}}_{p,q} = \Delta_p \Delta_q V^{\mathrm{ee}}(|\vec{r}_{\mathrm{atom}(p)} - \vec{r}_{\mathrm{atom}(q)}|) . $$

In [44]:
def calculate_interaction_matrix():
    """Return the electron-electron interaction energy matrix."""
    interaction_matrix = np.zeros((ndof,ndof))
    for orb_p in orbital_types:
        for orb_q in orbital_types:
            v_ee = eval_kernel( grad2(grad1(v_ee_kernel, orb_p), orb_q) )
            for atom_p,r_p in enumerate(atomic_coordinates):
                p = index(atom_p,orb_p)
                for atom_q,r_q in enumerate(atomic_coordinates):
                    q = index(atom_q,orb_q)
                    interaction_matrix[p,q] = v_ee(r_p, r_q)
    return interaction_matrix
                    
interaction_matrix = calculate_interaction_matrix()
print(interaction_matrix)

[[ 1.2e-01  0.0e+00  0.0e+00  0.0e+00  1.0e-01 -2.1e-03 -2.7e-03 -3.4e-03]
 [ 0.0e+00 -2.5e-01  0.0e+00  0.0e+00  2.1e-03  6.1e-04 -1.1e-04 -1.4e-04]
 [ 0.0e+00  0.0e+00 -2.5e-01  0.0e+00  2.7e-03 -1.1e-04  5.4e-04 -1.8e-04]
 [ 0.0e+00  0.0e+00  0.0e+00 -2.5e-01  3.4e-03 -1.4e-04 -1.8e-04  4.6e-04]
 [ 1.0e-01  2.1e-03  2.7e-03  3.4e-03  1.2e-01  0.0e+00  0.0e+00  0.0e+00]
 [-2.1e-03  6.1e-04 -1.1e-04 -1.4e-04  0.0e+00 -2.5e-01  0.0e+00  0.0e+00]
 [-2.7e-03 -1.1e-04  5.4e-04 -1.8e-04  0.0e+00  0.0e+00 -2.5e-01  0.0e+00]
 [-3.4e-03 -1.4e-04 -1.8e-04  4.6e-04  0.0e+00  0.0e+00  0.0e+00 -2.5e-01]]


### B. Slater-Koster tight-binding model

We will describe the kinetic energy of electrons using a simplified [Slater-Koster tight-binding method](https://en.wikipedia.org/wiki/Tight_binding). While it is compatible with very general functional forms, we will use Gaussians to simplify the model and its implementation. This part of the semiempirical model contains a total of 7 parameters: 2 orbital energies, $E_s$ and $E_p$, a length scale $r_{\mathrm{SK}}$ for interatomic interactions, and 4 parameters for the strength of these interactions between orbitals of different types and orientations: $t_{sss}$, $t_{sps}$, $t_{pps}$, and $t_{ppp}$:

In [15]:
energy_s = 0.2
energy_p = -0.591029
r_sk = 6.5
t_sss = 0.004
t_sps = 0.003
t_pps = 0.002
t_ppp = 0.001

The distance-dependent "hopping" matrix elements are then defined by

$$ t_{o,o'}^{\mathrm{SK}}(\vec{r}) = \exp(-r^2/r_{\mathrm{SK}}^2) \times \begin{cases}
 t_{sss} , & o = o' = s \\
 [\vec{o}' \cdot (\vec{r}/r_{\mathrm{SK}})] t_{sps}, & o = s \ \& \ o' \in \{p_x, p_y, p_z\} \\
 -[\vec{o} \cdot (\vec{r}/r_{\mathrm{SK}})] t_{sps} , & o' = s \ \& \ o \in \{p_x, p_y, p_z\} \\
 (r^2/r_{\mathrm{SK}}^2)\,(\vec{o} \cdot \vec{o}')  t_{pps} + [\vec{o} \cdot (\vec{r}/r_{\mathrm{SK}})] [\vec{o}' \cdot (\vec{r}/r_{\mathrm{SK}})] (t_{ppp} - t_{pps}), & o,o' \in \{p_x, p_y, p_z\}
 \end{cases} $$
 
where we are using $\vec{p}_x \equiv (1,0,0)$, $\vec{p}_y \equiv (0,1,0)$, and $\vec{p}_z \equiv (0,0,1)$ to simplify the notation. This mathematical structure describes the correct orientational dependence of translation-independent 1-body matrix elements between atomic orbitals without specifying anything except their orbital angular momentum. This project has multiple case-based formulas, and we will implement them using a code structure similar to each formula:

In [23]:
vec = { 'px':[1,0,0], 'py':[0,1,0], 'pz':[0,0,1] }

def hopping_matrix_element(o1,o2,r12):
    """Returns the hopping matrix element for a pair of orbitals of types o1 & o2 separated by a vector r12."""
    r12_rescaled = r12 / r_sk
    r12_squared = np.dot(r12_rescaled, r12_rescaled)
    ans = np.exp( -r12_squared )
    if o1 == 's' and o2 == 's':
        ans *= t_sss
    if o1 == 's' and o2 in p_orbitals:
        ans *= np.dot(vec[o2], r12_rescaled) * t_sps
    if o2 == 's' and o1 in p_orbitals:
        ans *= -np.dot(vec[o1], r12_rescaled)* t_sps
    if o1 in p_orbitals and o2 in p_orbitals:
        ans *= ( r12_squared * np.dot(vec[o1], vec[o2]) * t_pps
                 + np.dot(vec[o1], r12_rescaled) * np.dot(vec[o2], r12_rescaled) * (t_ppp - t_pps) )
    return ans

print("vec[px] =",vec['px'])
print("hopping test",hopping_matrix_element('s','px',np.array([1.0,0.0,0.0])))

vec[px] = [1, 0, 0]
hopping test 0.000450742738301092


The tight-binding contribution to the 1-body matrix elements is $t_{o,o'}^{\mathrm{SK}}$ for inter-atomic matrix elements and an orbital energy for intra-atomic matrix elements,

$$ h^{\mathrm{SK}}_{p,q} = \begin{cases}
 E_{\mathrm{orb}(p)} , & p = q \\
 t^{\mathrm{SK}}_{\mathrm{orb}(p),\mathrm{orb}(q)}(\vec{r}_{\mathrm{atom}(p)} - \vec{r}_{\mathrm{atom}(q)})
  , & \mathrm{atom}(p) \neq \mathrm{atom}(q) \\
  0 , & \mathrm{otherwise}
  \end{cases} $$


In [22]:
orbital_energy = { 's':energy_s, 'px':energy_p, 'py':energy_p, 'pz':energy_p }

def build_kinetic_matrix():
    """Returns a Hamiltonian matrix built using the available global information"""
    new_hamiltonian_matrix = np.zeros((ndof,ndof))
    for p in range(ndof):
        for q in range(ndof):
            if p == q:
                new_hamiltonian_matrix[p,q] = orbital_energy[orb(p)]
            if atom(p) != atom(q):
                r_pq = atomic_coordinates[atom(p)] - atomic_coordinates[atom(q)]
                new_hamiltonian_matrix[p,q] = hopping_matrix_element(orb(p), orb(q), r_pq)
    return new_hamiltonian_matrix

hamiltonian_matrix = build_hamiltonian_matrix()
print("hamiltonian matrix =\n",hamiltonian_matrix)

hamiltonian matrix =
 [[ 2.0e-01  0.0e+00  0.0e+00  0.0e+00  1.2e-03 -4.2e-04 -5.7e-04 -7.1e-04]
 [ 0.0e+00 -5.9e-01  0.0e+00  0.0e+00  4.2e-04  6.6e-04 -8.7e-05 -1.1e-04]
 [ 0.0e+00  0.0e+00 -5.9e-01  0.0e+00  5.7e-04 -8.7e-05  6.1e-04 -1.4e-04]
 [ 0.0e+00  0.0e+00  0.0e+00 -5.9e-01  7.1e-04 -1.1e-04 -1.4e-04  5.4e-04]
 [ 1.2e-03  4.2e-04  5.7e-04  7.1e-04  2.0e-01  0.0e+00  0.0e+00  0.0e+00]
 [-4.2e-04  6.6e-04 -8.7e-05 -1.1e-04  0.0e+00 -5.9e-01  0.0e+00  0.0e+00]
 [-5.7e-04 -8.7e-05  6.1e-04 -1.4e-04  0.0e+00  0.0e+00 -5.9e-01  0.0e+00]
 [-7.1e-04 -1.1e-04 -1.4e-04  5.4e-04  0.0e+00  0.0e+00  0.0e+00 -5.9e-01]]


### C. Multipole decomposition

To define $V_{p,q,r,s}$ based on the Coulomb matrix elements $V_{p,q}^{\mathrm{ee}}$, we need to define a mapping from products of atomic orbitals to a linear combination of terms in the multipole expansion of electronic charge. Because the atomic orbitals are normalized, their monopole coefficient with themselves is 1 (i.e. they have unit charge). Because of orthogonality, there is no monopole term for either intra-atomic or inter-atomic transitions between atomic orbitals. We will ignore dipole transitions between atoms, which corresponds to the [neglect of diatomic differential overlap](https://en.wikipedia.org/wiki/NDDO) (NDDO) approximation that is commonly used in semiempirical quantum chemistry. All that remains is the intra-atomic s-p transition, and we will use the final parameter of the model, $\alpha$, to define its dipole strength. These transformation rules are all contained in a tensor

$$ \chi_{p, q, r} = \begin{cases} 1, & \mathrm{orb}(p) = \mathrm{orb}(q) \ \& \ \mathrm{orb}(r) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\ 
 \alpha , & \mathrm{orb}(q) = \mathrm{orb}(r) \in \{p_x, p_y, p_z\} \ \& \ \mathrm{orb}(p) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\
 \alpha , & \mathrm{orb}(p) = \mathrm{orb}(r) \in \{p_x, p_y, p_z\} \ \& \ \mathrm{orb}(q) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\
 0, & \mathrm{otherwise} \end{cases} . $$

In [ ]:
ALPHA = 0.1

def atom_chi(o1,o2,o3):
    if o1 == o2 and o3 == 's':
        return 1.0
    elif o1 == o3 and o3 in p_set and o2 == 's':
        return ALPHA
    elif o2 == o3 and o3 in p_set and o1 == 's':
        return ALPHA
    else:
        return 0.0

The multipole expansion plays the same role as an auxiliary basis set in modern resolution-of-identity (RI) methods that are used to accelerate large quantum chemistry simulations. We can also use it to define the potential term to complete the 1-body matrix elements,

$$ h_{p,q} = h_{p,q}^{\mathrm{SK}} + \sum_{r} \chi_{p,q,r} V_{r}^{\mathrm{eI}}, $$

In [ ]:
for r in range(ndof):
    for orb_p in orbitals:
        for orb_q in orbitals:
            p = index(atom(r),orb_p)
            q = index(atom(r),orb_q)
            hamiltonian_matrix[p,q] += atom_chi(orb_p,orb_q,orb(r)) * potential_vector[r]
print(hamiltonian_matrix)

and the full tensor of 2-body Coulomb matrix elements,

$$ V_{p,q,r,s} = \sum_{t,u} \chi_{p,q,t} V_{t,u}^{\mathrm{ee}} \chi_{r,s,u} . $$

In [ ]:
def v_tensor(p,q,r,s):
    ans = 0.0
    if atom(p) != atom(q) or atom(r) != atom(s):
        return ans
    for orb_t in orbitals:
        t = index(atom(p),orb_t)
        for orb_u in orbitals:
            u = index(atom(q),orb_u)
            ans += atom_chi(orb(p),orb(q),orb_t) * interaction_matrix[t,u] * atom_chi(orb(r),orb(s),orb_u)
    return ans

Unlike the previous vectors and matrices, the $\chi$ and $V$ tensors are implemented as functions, analogous to "direct" quantum chemistry methods that recompute integrals on-the-fly. This saves quite a lot of memory at the expense of some redundant computation.

## 3. Hartree-Fock theory

Even with a simple model for $\hat{H}$, we cannot calculate its ground state energy exactly for more than a few Argon atoms. Instead, we will use the [Hartree-Fock approximation](https://en.wikipedia.org/wiki/Hartree–Fock_method), which restricts the ground-state wavefunction to a single Slater determinant. We will find the Slater determinant with the lowest total energy, but a more general wavefunction will usually have an even lower energy. In the next section, we will use many-body perturbation theory to improve our estimate of the total energy.

The central objects of Hartree-Fock theory are the 1-electron density matrix $\rho_{p,q}$ and the Fock matrix $f_{p,q}$. These two matrices depend on each other, which defines a nonlinear set of equations that we must solve iteratively, which is usually referred to as the self-consistent field (SCF) cycle. For this to converge, we must start from a reasonable initial guess for $\rho_{p,q}$. We will initialize it to the density matrix for isolated Argon atoms,

$$ \rho_{p,q}^{\mathrm{atom}} = \begin{cases} 
   1, & p = q \ \& \ \mathrm{orb}(p) \in \{ p_x, p_y, p_z \} \\
   0, & \mathrm{otherwise}
\end{cases} $$

In [ ]:
density_matrix = np.diag( np.tile([0,1,1,1],num_atoms) )
num_occ = num_atoms*3
print(density_matrix)

Because of spin symmetry, we use the same $\rho_{p,q}$ for each spin type, which reduces the sum over spin types in $\hat{H}$ to degeneracy pre-factors. Thus, half of the electrons are spin-up and half are spin-down.

The Fock matrix is defined by the density matrix,

$$ f_{p,q} = h_{p,q} + \sum_{r,s} ( 2 V_{p,q,r,s} - V_{p,s,r,q} )\rho_{r,s} , $$

In [ ]:
def build_fock_matrix(density_matrix0):
    new_fock_matrix = hamiltonian_matrix.copy()
    for p in range(ndof):
        for r in range(ndof):
            for orb_q in orbitals:
                for orb_s in orbitals:
                    q = index(atom(p),orb_q)
                    s = index(atom(r),orb_s)
                    new_fock_matrix[p,q] += 2.0*v_tensor(p,q,r,s)*density_matrix0[r,s]
        for q in range(ndof):
            for orb_r in orbitals:
                for orb_s in orbitals:
                    r = index(atom(q),orb_r)
                    s = index(atom(p),orb_s)
                    new_fock_matrix[p,q] -= v_tensor(p,s,r,q)*density_matrix0[r,s]
    return new_fock_matrix

the Fock matrix defines the matrix of molecular orbitals $\phi_{i,j}$ as its eigenvectors,

$$ \sum_{q} f_{p,q} \phi_{q,r} = E_r \phi_{p,r}, $$

and the density matrix is defined by the occupied orbitals (we occupy the lowest-energy orbitals with all of the electrons available),

$$ \rho_{p,q} = \sum_{i \in \mathrm{occ}} \phi_{p,i} \phi_{q,i}. $$

In [ ]:
def build_density_matrix(fock_matrix0):
    orbital_energy, orbital_matrix = np.linalg.eigh(fock_matrix0)
    occ_matrix = orbital_matrix[:,:num_occ]
    return occ_matrix @ occ_matrix.T

To calculate a $\rho_{p,q}$ and $f_{p,q}$ that are consistent with each other, we mix in a fraction of the new density matrix with the old density matrix and re-calculate them until convergence. This is a crude strategy for an SCF cycle, but it works for weakly interacting atoms like Argon:

In [ ]:
MAX_SCF_ITERATIONS = 100
MIXING_FRACTION = 0.25
CONVERGENCE_TOLERANCE = 1e-4
for i in range(MAX_SCF_ITERATIONS):
    fock_matrix = build_fock_matrix(density_matrix)
    new_density_matrix = build_density_matrix(fock_matrix)

    error_norm = np.linalg.norm( density_matrix - new_density_matrix )
    print(error_norm)
    if error_norm < CONVERGENCE_TOLERANCE: break

    density_matrix = (MIXING_FRACTION*new_density_matrix
                      + (1.0 - MIXING_FRACTION)*density_matrix)

Once we have a converged density matrix, the Hartree-Fock total energy is defined as

$$ \begin{align}
  E_{\mathrm{HF}} &= \sum_{i < j} \frac{Z^2}{|\vec{r}_i - \vec{r}_j|} + 2 \sum_{p,q} h_{p,q} \rho_{p,q}
  + \sum_{p,q,r,s} V_{p,q,r,s} ( 2 \rho_{p,q} \rho_{r,s} - \rho_{p,s} \rho_{r,q} ) \\
  & = \sum_{i < j} \frac{Z^2}{|\vec{r}_i - \vec{r}_j|} + \sum_{p,q} (h_{p,q} + f_{p,q}) \rho_{p,q}
  \end{align} $$

In [ ]:
energy_hf = energy_ion + np.einsum('pq,pq',hamiltonian_matrix + fock_matrix,density_matrix)
print(energy_hf)

Once we have converged the SCF cycle, we can finish splitting up the orbitals and their energies into occupied and unoccupied/virtual to prepare for the next set of calculations:

In [ ]:
orbital_energy, orbital_matrix = np.linalg.eigh(fock_matrix)
occ_energy = orbital_energy[:num_occ]
virt_energy = orbital_energy[num_occ:]
occ_matrix = orbital_matrix[:,:num_occ]
virt_matrix = orbital_matrix[:,num_occ:]

## 4. 2nd-order Moller-Plesset (MP2) perturbation theory

The Hartree-Fock approximation does not describe the London dispersion interaction between Argon atoms. We must use many-body perturbation theory to improve our approximation to the ground state of $\hat{H}$. Thankfully, we only need to use the first correction beyond Hartree-Fock theory, which is [second-order Moller-Plesset (MP2) perturbation theory](https://en.wikipedia.org/wiki/Møller–Plesset_perturbation_theory). It is simple, but computationally expensive. The computational bottleneck is the transformation of the tensor $V_{p,q,r,s}$ from the atomic orbital basis to the molecular orbital basis,

$$ \tilde{V}_{a,i,b,j} = \sum_{p,q,r,s} \phi_{p,a} \phi_{q,i} \phi_{r,b} \phi_{s,j} V_{p,q,r,s}. $$

where we are restricting some indices to occupied orbitals ($i$ and $j$) and some indices to the unoccupied "virtual" orbitals ($a$ and $b$). This use of specific orbital labels to denote a restriction of indices is standard notation in quantum chemistry. We can be a little bit clever and use the factored form of $V_{p,q,r,s}$ to reduce our intermediate memory usage,

$$ \begin{align}
   \tilde{\chi}_{a,i,p} &= \sum_{q,r} \phi_{q,a} \phi_{r,i} \chi_{q,r,p} \\
   \tilde{V}_{a,i,b,j} &= \sum_{p,q} \tilde{\chi}_{a,i,p} V_{p,q}^{\mathrm{ee}} \tilde{\chi}_{b,j,q}
   \end{align} $$

In [ ]:
chi2_tensor = np.zeros((ndof,ndof,ndof))
for p in range(ndof):
    for q in range(ndof):
        for r in range(ndof):
            if atom(p) == atom(q) and atom(q) == atom(r):
                chi2_tensor[p,q,r] = atom_chi(orb(p),orb(q),orb(r))

chi2_tensor = np.einsum('qa,ri,qrp',virt_matrix,occ_matrix,chi2_tensor,optimize=True)
v2_tensor = np.einsum('aip,pq,bjq->aibj',chi2_tensor,interaction_matrix,chi2_tensor,optimize=True)

In this simple implementation, we are not taking full advantage of the sparisty of the initial $\chi$ tensor. Unlike the $V$ tensor, the $\tilde{V}$ tensor is constructed explicitly and stored in memory. The MP2 correction to the total energy is then just a large summation over previously computed and stored quantities,

$$ E_{\mathrm{MP2}} = - \sum_{a,b \in \mathrm{virt}} \sum_{i,j \in \mathrm{occ}} \frac{2 \tilde{V}_{a,i,b,j}^2 - \tilde{V}_{a,i,b,j} \tilde{V}_{a,j,b,i}}{E_a + E_b - E_i - E_j} . $$

In [ ]:
energy_mp2 = 0.0
num_virt = num_atoms
for a in range(num_virt):
    for b in range(num_virt):
        for i in range(num_occ):
            for j in range(num_occ):
                energy_mp2 -= ( (2.0*v2_tensor[a,i,b,j]**2 - v2_tensor[a,i,b,j]*v2_tensor[a,j,b,i])
                                /(virt_energy[a] + virt_energy[b] - occ_energy[i] - occ_energy[j]) )
print(energy_mp2)

The MP2 bottleneck in the formation of $\tilde{V}$ from $\tilde{\chi}$ and $V^{\mathrm{ee}}$ scales as $O(n^5)$ operations for $n$ atoms because of 4 free tensor indices and 1 summed tensor index once the contraction has been optimally arranged into intermediate operations.

## (Extra Credit) 5. Localized perturbation theory

An important area of ongoing research is the use of localization to accelerate QM calculations. Noble gases are an extreme example that enable localization to be applied very aggressively. If we will treat $t_{p,q}^{\mathrm{SK}}$ as a perturbation, then the Hartree-Fock equations are solved by our initial atomic guess for $\rho_{p,q}$. The energy contributions from pairs of atoms decays rapidly with distance, and we can ignore pairs of well-separated atoms.

In [ ]:
max_distance = 15.0
nearby_pairs = [ (i,j) for i in range(num_atoms)
                       for j in range(num_atoms)
                       if i < j
                       and np.linalg.norm(coordinates[i]-coordinates[j]) <= max_distance ]
print(nearby_pairs)

The localized Hartree-Fock energy can be computed from the independent-atom solutions and a localized sum of pairwise atomic contributions that combine ion-ion, electron-ion, and electron-electron Coulomb energy contributions.

In [ ]:
v_ei_onsite = eval_kernel(v_ei_kernel)([0,0,0],[0,0,0])
v_ee_onsite = eval_kernel(v_ee_kernel)([0,0,0],[0,0,0])
energy_hf_local = num_atoms*Z*(E_P + v_ei_onsite + 0.5*Z*v_ee_onsite)

v_atom = eval_kernel(v_ii_kernel + Z*v_ei_kernel + (Z**2)*v_ee_kernel)
for pair in nearby_pairs:
    r1 = coordinates[pair[0]]
    r2 = coordinates[pair[1]]
    energy_hf_local += v_atom(r1,r2)

In addition to a localized version of the MP2 energy, we must also account for the perturbative correction to the Hartree-Fock energy upon introducing $t_{p,q}^{\mathrm{SK}}$,

$$ E_{\Delta HF} = - 2 \sum_{a \in \mathrm{virt}} \sum_{i \in \mathrm{occ}} \frac{(t_{a,i}^{\mathrm{SK}})^2}{E_a - E_i} . $$

The localized MP2 correction is identical to the standard correction, but the $\tilde{V}$ tensor can be replaced by the $V$ tensor and we can avoid storing and transforming an explicit tensor. In the atomic solution, the $p$ orbitals are occupied and the $s$ orbitals are virtual.

In [ ]:
energy_hf_delta = 0.0
for pair in nearby_pairs:
    r1 = coordinates[pair[0]]
    r2 = coordinates[pair[1]]
    for o in p_set:
        energy_hf_delta -= 2.0*(t_sk('s',o,r1-r2)**2 + t_sk(o,'s',r1-r2)**2) / (E_S - E_P)

energy_mp2_local = 0.0
for o1 in p_set:
    for o2 in p_set:
        a = index(0,'s')
        b = index(0,'s')
        i = index(0,o1)
        j = index(0,o2)
        energy_mp2_local -= num_atoms*((2.0*v_tensor(a,i,b,j)**2
                                        - v_tensor(a,i,b,j)*v_tensor(a,j,b,i))
                                        / (2.0*E_S - 2.0*E_P))
for pair in nearby_pairs:
    for o1 in p_set:
        for o2 in p_set:
            a = index(pair[0],'s')
            b = index(pair[1],'s')
            i = index(pair[0],o1)
            j = index(pair[1],o2)
            energy_mp2_local -= 4.0*v_tensor(a,i,b,j)**2 / (2.0*E_S - 2.0*E_P)

print(energy_hf_local,energy_hf_delta,energy_mp2_local)

Except for the formation of the nearby atomic pairs list, this entire localized calculation scales as $O(n)$ operations for $n$ atoms. This is an example of a QM calculation that has been stripped down to the point where it can be competitive in cost with simple interatomic potentials.

## (Extra Credit) 6. Semiempirical model parameterization

During the summer school, you will be developing this base QM project into a more fleshed-out piece of software.

### A. Function encapsulation

It is good programming practice to isolate segments of code with common functionality into functions. A code written in terms of well-designed, nested functions is usually much more readable than the monolithic code block presented here. In particular, we would like this project's functionality to be accessible as a function that inputs coordinates and outputs a total energy. The standard and localized calculations are distinct approximations with very different costs, and they should be independently accessible with a similar interface.

### B. Testing & model validation

### C. Object-oriented programming

### D. C++ refactoring

### E. "Extra credit"

Calculate the list of nearby atomic pairs with a linear-scaling algorithm. Fine-tune the localization cutoff radius (<tt>max_distance</tt>) to better balance cost versus accuracy.